# Lab 10

In [2]:
!#git clone -b poleval/task3 https://github.com/n-waves/fastai --depth 1 

#### Imports

In [3]:
from typing import Dict, List, Tuple, Optional
from collections import namedtuple
import os

In [4]:
from functional import seq, pseq
from functional.streams import Sequence
from IPython.core.display import HTML

def display_seq(sequence:Sequence,rows:int)-> None:
        sequence._repr_html_= lambda :sequence.tabulate(rows,tablefmt='html')
        display(sequence)
        sequence._repr_html_= lambda :sequence.tabulate(10,tablefmt='html')

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import functools
def compose(*functions):
    def compose2(f, g):
        return lambda x: f(g(x))
    return functools.reduce(compose2, functions, lambda x: x)

## Setup

In [7]:
!tree blobs.bak/work/up_low50k/

blobs.bak/work/up_low50k/
├── models
│   └── fwd_v50k_finetune_lm_enc.h5
└── tmp
    ├── sp-50k.model
    └── sp-50k.vocab

2 directories, 3 files


In [8]:
fastai_model_path = "./blobs.bak/work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5"
sentencepiece_model_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.model"
sentencepiece_vocab_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.vocab"

In [9]:
def exists_or_ex(path):
    file = open(path,"r")
    file.close()
_ = [exists_or_ex(path) for path in [fastai_model_path, sentencepiece_model_path, sentencepiece_vocab_path]]

## Sentencepiece

In [10]:
import sentencepiece as spm
spm_processor = spm.SentencePieceProcessor()
spm_processor.Load(sentencepiece_model_path)

spm_processor.SetEncodeExtraOptions("bos:eos")
spm_processor.SetDecodeExtraOptions("bos:eos")
# spm_processor.LoadVocabulary(sentencepiece_vocab_path,threshold= 100)

True

In [11]:
# spm_processor.encode_as_pieces("Ala ma kota i kot alę")


In [12]:
for n in range(5):
    print(spm_processor.SampleEncodeAsPieces('New York', -1, 0.1))

[b'<s>', b'\xe2\x96\x81New', b'\xe2\x96\x81Y', b'o', b'rk', b'</s>']
[b'<s>', b'\xe2\x96\x81New', b'\xe2\x96\x81Yo', b'rk', b'</s>']
[b'<s>', b'\xe2\x96\x81New', b'\xe2\x96\x81Y', b'or', b'k', b'</s>']
[b'<s>', b'\xe2\x96\x81New', b'\xe2\x96\x81Yo', b'rk', b'</s>']
[b'<s>', b'\xe2\x96\x81N', b'e', b'w', b'\xe2\x96\x81York', b'</s>']


In [13]:
from fastai.text import *

In [14]:
! head -n ./blobs.bak/

head: invalid number of lines: ‘./blobs.bak/’


In [15]:
# wgts = torch.load(fastai_model_path, map_location=lambda storage, loc: loc)
wgts = torch.load(fastai_model_path, map_location='cuda:0')
# enc_wgts = to_np(wgts['0.encoder.weight'])
# row_m = enc_wgts.mean(0)



In [16]:
def get_lm(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token, bidir=False,
           tie_weights=True, qrnn=False):
    rnn_enc = MultiBatchRNN(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token=pad_token, bidir=bidir, qrnn=qrnn)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(n_tok, emb_sz, 0, tie_encoder=enc))

In [17]:
# enc_wgts = to_np(wgts['encoder.weight']) #I think it expects  wgts top have 0.endoder.weight
# row_m = enc_wgts.mean(0)
torch.cuda.set_device(0)

UNK_ID = 0
PAD_ID = 1
BOS_ID = 2
EOS_ID = 3
UP_ID  = 4
bs=22

lm_path=  fastai_model_path

# em_sz,nh,nl = 400,1150,3  # that should be the case
em_sz,nh,nl = 400 ,1150,4

bptt=5
vs = len(spm_processor)

lm = get_lm(bptt, 1000000, vs, em_sz, nh, nl, PAD_ID)
lm = to_gpu(lm)
load_model(lm[0],lm_path)
lm.reset()
lm.eval()


SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(50000, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(50000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
      (3): LockedDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=50000, bias=False)
    (dropout): LockedDropout()
  )
)

In [18]:
class LMTextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]

    def __len__(self): return len(self.x)



In [19]:
from torchtext import data,vocab

### From infer

In [20]:
inpucik = "Mama bardzo dobrze wydała swoje"
test_ids = [np.array(spm_processor.encode_as_ids(inpucik))]
test_ids

[array([    2,   618,     8,    91,   431, 11111,   344,     3])]

In [21]:
test_ds = LMTextDataset(test_ids)
test_samp = SortSampler(test_ids, key=lambda x: len(test_ids[x]))
test_dl = DataLoader(test_ds, bs, transpose=True, transpose_y=True, num_workers=1, pad_idx=PAD_ID, sampler=test_samp, pre_pad=False)
x,y = list(test_dl)[0]
x2 = [int(i) for i in x]
# y2 = [int(i) for i in y]
spm_processor.decode_ids(x2)

'Mama bardzo dobrze wydała swoje'

In [28]:
def next_word(ss,model,toks_at_once):
    ids = spm_processor.encode_as_ids(ss)
    return spm_processor.decode_ids(
        next_tokens(ids,model,toks_at_once)
    )

def next_tokens(ids_,model,toks_at_once): #toks_at_once wont be used
    ids = [np.array(ids_)]
    test_ds = LMTextDataset(ids)
    test_samp = SortSampler(ids, key=lambda x: len(test_ids[x]))
    dl = DataLoader(test_ds, bs, transpose=True, transpose_y=True, num_workers=1, pad_idx=PAD_ID, sampler=test_samp, pre_pad=False)
    
    tensor1 = None
    model.reset() # todo:bcm - do or dont'?
    with no_grad_context():
        for (x, y) in dl:
            tensor1= model(x)
    p= tensor1[0]
    print(p.size())
#     r = torch.multinomial(p[-1].exp(), toks_at_once)
    r = [seq(i for i in p[-1]).map(lambda x: float(x)).enumerate().max_by(lambda id_val: id_val[1])[0]]
    
    predicted_ids = [int(r_) for r_ in r]
    return predicted_ids

In [29]:
# def next_words_stm(ss,lm,pred_len, input_tokens = 8):
#     result = spm_processor.encode_as_ids(ss)
    
#     for i  in range(pred_len):
#         res_i = next_tokens(result[:input_tokens],lm,1)
#         result = result + res_i
#     return spm_processor.decode_ids(result)

# def get_next_n(inp, n):
#     res = inp
#     for i in range(n):
#         c = get_next(inp)
#         res += c
#         inp = inp[1:]+c
#         if c == '.': break
#     return res

In [24]:
def next_words_good(ss,lm, n_words, toks_at_once=1):
    wip = spm_processor.encode_as_ids(ss)
    for i  in range(n_words):
        ala = wip
        wip =  ala + next_tokens(wip,lm,toks_at_once)
    return spm_processor.decode_ids(wip)

In [25]:
def next_words_bad(ss,lm, n_words, toks_at_once=1):
    wip = ss
    for i  in range(n_words):
        wip =  wip + " "+ next_word(wip,lm,toks_at_once)
    return wip

In [32]:
next_words_good("Kraków to miasto",lm,10)

torch.Size([4, 50000])
torch.Size([5, 50000])
torch.Size([6, 50000])
torch.Size([7, 50000])
torch.Size([8, 50000])
torch.Size([9, 50000])
torch.Size([10, 50000])
torch.Size([11, 50000])
torch.Size([12, 50000])
torch.Size([13, 50000])


'Kraków to miasto , W W tymi roku mieście w było'

In [33]:
next_words_bad("Matka zrobiła",lm,10)

torch.Size([3, 50000])
torch.Size([4, 50000])
torch.Size([5, 50000])
torch.Size([6, 50000])
torch.Size([7, 50000])
torch.Size([8, 50000])
torch.Size([9, 50000])
torch.Size([10, 50000])
torch.Size([11, 50000])
torch.Size([12, 50000])


'Matka zrobiła to , co zrobiła , a ja nie wiedziała m'

In [26]:
sentence = "Zły dom"
en = 30
print("")
print("BAd en,1")
print(next_words_bad(sentence,lm,en,1))
print("")
print("Bad 1,en")
print(next_words_bad(sentence,lm,1,en)) # poor
print("")
print("Good en, 1")
print(next_words_good(sentence,lm,en,1))
print("")
print("Good 1, en")
print(next_words_good(sentence,lm,1,en)) # poor
print("")
print("stm")
print(next_words_stm(sentence,lm,en,input_tokens= 5))


BAd en,1
torch.Size([5, 50000])
torch.Size([6, 50000])
torch.Size([7, 50000])
torch.Size([8, 50000])
torch.Size([9, 50000])
torch.Size([11, 50000])
torch.Size([12, 50000])
torch.Size([13, 50000])
torch.Size([14, 50000])
torch.Size([15, 50000])
torch.Size([16, 50000])
torch.Size([17, 50000])
torch.Size([18, 50000])
torch.Size([19, 50000])
torch.Size([20, 50000])
torch.Size([21, 50000])
torch.Size([22, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
torch.Size([23, 50000])
Zły dom , w którym mieszka sz o to , że nie ma w nim miejsca na odpoczynek .             

Bad 1,en
torch.Size([5, 50000])
Zły dom ,

Good en, 1
torch.Size([5, 50000])
torch.Size([6, 50000])
torch.Size([7, 50000])
torch.Size([8, 50000])
torch.Size([9, 50000])
torch.Size([10,

In [27]:
for i in range(3,10):
    print(next_words_stm("Te zabawki należą do",lm,20,input_tokens=i))

torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
torch.Size([2, 50000])
Te zabawki należą dożżżżżżżżżżżżżżżżżżżż
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
torch.Size([3, 50000])
Te zabawki należą doiiiiiiiiiiiiiiiiiii

## Playground

In [241]:
inpucik = "Mama bardzo dobrze wydała swoje ulubione "
test_ids = [np.array(spm_processor.encode_as_ids(inpucik))]
test_ids

[array([    2,   618,     8,    91,   431, 11111,   344, 30517,     3])]

In [242]:
test_ds = LMTextDataset(test_ids)
test_samp = SortSampler(test_ids, key=lambda x: len(test_ids[x]))
test_dl = DataLoader(test_ds, bs, transpose=True, transpose_y=True, num_workers=1, pad_idx=PAD_ID, sampler=test_samp, pre_pad=False)
x,y = list(test_dl)[0]
x2 = [int(i) for i in x]
# y2 = [int(i) for i in y]
spm_processor.decode_ids(x2)

'Mama bardzo dobrze wydała swoje ulubione'

In [243]:
def getp():
    dl = test_dl
    loss = 0.0
    tensor1 = None
    lm.reset()
    with no_grad_context():
        for (x, y) in tqdm(dl):
            targets = y.view(-1)
            preds = lm(x)[0]
            tensor1= lm(x)
            not_pads = targets != PAD_ID
            ce = F.cross_entropy(preds[not_pads], targets[not_pads], reduction='sum')
            loss += ce
    loss
    p= tensor1[0]
    print(p[0].size())
    print ([type(i) for i in tensor1])
    return p

In [244]:
def get_prediction(n):
    p = getp()
    print(p.size())
    
#     for pi in p:
#         r = torch.multinomial(pi.exp(), n)
#         predicted_ids = [int(r_) for r_ in r]
#         print(spm_processor.decode_ids(predicted_ids))
#         print("")
#         print("")

    r = torch.multinomial(p[-1].exp(), 10)
    print(r)
    print([p[-1][i] for i in r])
    predicted_ids = [int(r_) for r_ in r]
    return spm_processor.decode_ids(predicted_ids)

In [245]:
inpucik

'Mama bardzo dobrze wydała swoje ulubione '

In [246]:
p = getp()

100%|██████████| 1/1 [00:00<00:00, 14.31it/s]
torch.Size([50000])
[<class 'torch.Tensor'>, <class 'list'>, <class 'list'>]


In [247]:
from functional import seq

In [248]:
seq(i for i in p[-1]).map(lambda x: float(x)).enumerate().max_by(lambda id_val: id_val[1])

[4838, 4.52532958984375]

In [250]:
spm_processor.decode_ids([4838])

'piosenki'

In [215]:
get_prediction(20)
np.array(p[-1])

100%|██████████| 1/1 [00:00<00:00, 16.97it/s]
torch.Size([50000])
[<class 'torch.Tensor'>, <class 'list'>, <class 'list'>]
torch.Size([7, 50000])
tensor([48758, 47070, 15515, 38977, 49387, 24972,  8005,  3566, 10795, 17061],
       device='cuda:0')
[tensor(-0.4503, device='cuda:0'), tensor(-0.5835, device='cuda:0'), tensor(-0.0726, device='cuda:0'), tensor(-2.5826, device='cuda:0'), tensor(-0.7917, device='cuda:0'), tensor(2.1340, device='cuda:0'), tensor(1.7828, device='cuda:0'), tensor(-4.4941, device='cuda:0'), tensor(-0.9323, device='cuda:0'), tensor(-0.6776, device='cuda:0')]


TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [187]:
# lm.predict(text="ala ma kota", n_words=10, temperature=0.75) works in newer version

# Dark  place:

In [101]:
import fastai
from fastai import learner

In [102]:
learner.load_model(lm,"")

FileNotFoundError: [Errno 2] No such file or directory: ''

In [103]:
wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))


drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.metrics = [accuracy]
learner.freeze_to(-1)

learner.model.load_state_dict(wgts)

lr=1e-2
lrs = lr

learner



NameError: name 'md' is not defined

# old things

In [ ]:
from fastai.torch_core import *
from fastai.basic_data import *
from fastai.callback import *
from fastai.data_block import *
from fastai.utils.ipython import gpu_mem_restore
import inspect
from fastprogress.fastprogress import format_time, IN_NOTEBOOK
from time import time
from fastai.sixel import plot_sixel

def load_that_learner(torch_thing,**db_kwargs):
    "Load a `Learner` object saved with `export_state` in `path/file` with empty data, optionally add `test` and load on `cpu`. `file` can be file-like (file or buffer)"
    state = torch_thing
    model = state.pop('model')
    src = LabelLists.load_state(path, state.pop('data'))
    if test is not None: src.add_test(test)
    data = src.databunch(**db_kwargs)
    cb_state = state.pop('cb_state')
    clas_func = state.pop('cls')
    res = clas_func(data, model, **state)
    res.callback_fns = state['callback_fns'] #to avoid duplicates
    res.callbacks = [load_callback(c,s, res) for c,s in cb_state.items()]
    return res

In [ ]:
load_that_learner(wgts)

In [ ]:
LM_PATH = "./blobs.bak/work/up_low50k/models"
LM_FILE = "fwd_v50k_finetune_lm_enc.h5"
model = load_learner(LM_PATH,LM_FILE,load_on='cpu')

In [ ]:
help(load_learner)

In [ ]:
help(get_language_model)